In [ ]:
# --- COLAB SETUP START ---
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # 1. Clone the repository
    !git clone https://github.com/google-gemini/gemini-fullstack-langgraph-quickstart
    %cd gemini-fullstack-langgraph-quickstart/backend

    # 2. Prepare Environment (Resolving Conflicts)
    import sys
    print("Uninstalling conflicting pre-installed packages...")
    !pip uninstall -y google-ai-generativelanguage tensorflow grpcio-status

    # Pre-install PyTorch Nightly if Python 3.13+ is detected
    if sys.version_info >= (3, 13):
        print("Detected Python 3.13+. Installing PyTorch Nightly...")
        !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

    # 3. Install Backend
    !pip install .

    # 4. Set API Key
    import os
    from google.colab import userdata
    try:
        os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
    except:
        print("Please enter your Gemini API Key:")
        os.environ["GEMINI_API_KEY"] = input()
# --- COLAB SETUP END ---

# Deep Research Agent Demo

This notebook demonstrates the Deep Research capabilities, including the planning and research loop.

In [ ]:
# Setup environment
import os
import sys
from dotenv import load_dotenv

sys.path.append(os.path.abspath("../backend/src"))
load_dotenv("../backend/.env")

## 1. Research Planning Mode

Demonstrating how the agent creates a plan before executing research.

In [ ]:
from agent.graph import graph
from langchain_core.messages import HumanMessage

async def run_planning_demo():
    # Enable planning mode via configuration
    config = {
        "configurable": {
            "thread_id": "demo_planning_1",
            "require_planning_confirmation": True
        }
    }
    
    input_text = "Research the latest advancements in solid state batteries from 2024."
    print(f"Request: {input_text}\n")
    
    # Send initial request to trigger planning
    async for event in graph.astream_events(
        {"messages": [HumanMessage(content=input_text)]},
        config,
        version="v2"
    ):
        if event["event"] == "on_chat_model_stream":
            # Print streaming output (likely the plan)
            print(event["data"]["chunk"].content, end="", flush=True)

await run_planning_demo()